In [2]:
from math import log2
import pandas as pd

# calculate the entropy for the split in the dataset
def entropy(class0, class1):
    if  class0 == 0:
        log_c1 = 0
        log_c2 = log2(class1)
    elif class1 == 0:
        log_c2 = 0
        log_c1 = log2(class0)
    else:    
        log_c1 = log2(class0)
        log_c2 = log2(class1)

    return -(class0 * log_c1 + class1 * log_c2)

def calculateRootAttribute(titanic_data):
    #print(titanic_data.shape[0])
    #calculate H(Li) for the last column
    true, false = titanic_data["Survival"].value_counts()


    tot_length = titanic_data.shape[0]
    #print(tot_length)

    true_split = true / tot_length
    false_split = false / tot_length
    tot_entropy = entropy(true_split, false_split)
    print("Total Entropy: ", tot_entropy, "\n") 

    titanic_data_iter = titanic_data.drop(labels="Survival", axis = 1)


    ig_gains_list = []
    #calculate IG for each attribute
    for column in titanic_data_iter:
        #get all the unique classifiers in the column
        classifiers = titanic_data[column].unique()
        print("Column: ", column)
        entropies_list = []
        #print(classifiers)
        for item in classifiers:
            subset = titanic_data.loc[titanic_data[column] == item]
            #print(subset)
            length = subset.shape[0]
            true_bool_series = subset.apply(lambda x: True if x['Survival'] == 'T' else False, axis = 1)
            true = len(true_bool_series[true_bool_series == True].index)
            false = len(true_bool_series[true_bool_series == False].index)
            #print(item," true: ", true)
            #print(item," false: ", false)
            #print(length)
            true_split = true / length
            false_split = false / length
            #print(true_split)
            attr_entropy = entropy(true_split, false_split)
            print("Entropy for ", item, ": ", attr_entropy)
            split_length = length/tot_length
            #print("Split length: ", split_length)
            entropies_list.append((attr_entropy, split_length ))
            #print(entropies_list)

        #ig gain for the column
        split_entropy_value = 0.0
        for item in entropies_list:
            split_entropy_value += (item[0] * item[1])
            #print(item[0], " ", item[1])
        ig_gain = tot_entropy - split_entropy_value
        ig_gains_list.append((ig_gain, column))
        print("\nIg gain for ", column, ": ", ig_gain)

    #Get root attribute
    temp_max = -1
    ans = ''
    for item in ig_gains_list:
        if item[0] > temp_max:
            ans = item[1]
            temp_max = item[0]

    print("The root attribute is: ", ans, " with IG: ", temp_max)


if __name__ == "__main__":
    titanic_data = pd.read_csv("hw2q1.csv")
    titanic_data.head()
    #options = ['Upper']
    #titanic_data = titanic_data.loc[titanic_data["Pclass"].isin(options)]

    #options = ['Expensive']
    #titanic_data = titanic_data.loc[titanic_data["Fare"].isin(options)]
    # print(titanic_data)
    # calculateRootAttribute(titanic_data)



        

   Pclass     Sex    Embarked       Fare Survival
0   Upper    Male   Cherbourg  Expensive        F
1   Upper  Female   Cherbourg  Expensive        T
3   Upper  Female  Queenstown  Expensive        T
9   Upper    Male   Cherbourg  Expensive        T
10  Upper    Male   Cherbourg  Expensive        T
12  Upper    Male  Queenstown      Cheap        F
15  Upper    Male   Cherbourg      Cheap        T
Total Entropy:  0.863120568566631 

Column:  Pclass
Entropy for  Upper :  0.863120568566631

Ig gain for  Pclass :  0.0
Column:  Sex
Entropy for  Male :  0.9709505944546686
Entropy for  Female :  -0.0

Ig gain for  Sex :  0.16958442967043919
Column:  Embarked
Entropy for  Cherbourg :  0.7219280948873623
Entropy for  Queenstown :  1.0

Ig gain for  Embarked :  0.061743357932800724
Column:  Fare
Entropy for  Expensive :  0.7219280948873623
Entropy for  Cheap :  1.0

Ig gain for  Fare :  0.061743357932800724
The root attribute is:  Sex  with IG:  0.16958442967043919
